<a href="https://colab.research.google.com/github/GodstimeEdet/5G-path-loss-prediction-using-CNN/blob/main/pathLossPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
# Navigate to the folder containing the images
os.chdir('/content/drive/MyDrive/data/train/')

In [ ]:
# List files in the folder
files = os.listdir()


In [ ]:
# Filter image files
image_files = [file for file in files if file.endswith(('.png', '.jpg', '.jpeg'))]


In [ ]:
# Load images
loaded_images = []
for image_file in image_files:
    try:
        # Load image using OpenCV
        image_path = os.path.join('/content/drive/MyDrive/data/train/', image_file)
        image = cv2.imread(image_path)

        if image is not None:
            loaded_images.append(image)
        else:
            print(f"Error: Unable to load image '{image_file}'")
    except Exception as e:
        print(f"Error: Exception occurred while loading image '{image_file}': {e}")

# Check if images were loaded successfully
if len(loaded_images) > 0:
    print(f"{len(loaded_images)} images loaded successfully!")
    # Perform further processing or analysis with the loaded images

50 images loaded successfully!


In [ ]:
loaded_images

[array([[[ 65,  99,  88],
         [ 67, 101,  90],
         [ 68, 102,  91],
         ...,
         [ 55,  72,  69],
         [ 58,  75,  72],
         [ 64,  81,  78]],
 
        [[ 58,  93,  79],
         [ 63,  98,  84],
         [ 66, 101,  87],
         ...,
         [ 52,  71,  68],
         [ 54,  71,  68],
         [ 54,  73,  70]],
 
        [[ 47,  83,  67],
         [ 56,  92,  76],
         [ 64, 100,  84],
         ...,
         [ 55,  76,  74],
         [ 56,  74,  73],
         [ 53,  74,  72]],
 
        ...,
 
        [[198, 205, 214],
         [176, 183, 192],
         [180, 189, 198],
         ...,
         [107, 111, 106],
         [105, 108, 106],
         [ 88,  91,  89]],
 
        [[169, 177, 184],
         [149, 157, 164],
         [155, 163, 170],
         ...,
         [106, 106, 100],
         [108, 107, 103],
         [100, 101,  97]],
 
        [[149, 157, 164],
         [143, 151, 158],
         [136, 144, 151],
         ...,
         [109, 107,  99],
  

In [ ]:
import cv2

def preprocess_image(image_path, target_size=(224, 224)):
    try:
        # Load the image using OpenCV
        image = cv2.imread(image_path)

        # Check if the image is loaded successfully
        if image is None:
            print(f"Error: Unable to load image '{image_path}'")
            return None

        # Resize the image to the target size
        image = cv2.resize(image, target_size)

        # Normalize the pixel values to be between 0 and 1
        image = image.astype("float32") / 255.0

        # Convert the image to RGB format (if it's not already in RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        return image
    except Exception as e:
        print(f"Error: Exception occurred while preprocessing image '{image_path}': {e}")
        return None

# Example usage:
image_path = loaded_images
preprocessed_image = preprocess_image(image_path)

if preprocessed_image is not None:
    print("Image preprocessed successfully!")
    # Perform further processing or analysis with the preprocessed image


Error: Exception occurred while preprocessing image '[array([[[ 65,  99,  88],
        [ 67, 101,  90],
        [ 68, 102,  91],
        ...,
        [ 55,  72,  69],
        [ 58,  75,  72],
        [ 64,  81,  78]],

       [[ 58,  93,  79],
        [ 63,  98,  84],
        [ 66, 101,  87],
        ...,
        [ 52,  71,  68],
        [ 54,  71,  68],
        [ 54,  73,  70]],

       [[ 47,  83,  67],
        [ 56,  92,  76],
        [ 64, 100,  84],
        ...,
        [ 55,  76,  74],
        [ 56,  74,  73],
        [ 53,  74,  72]],

       ...,

       [[198, 205, 214],
        [176, 183, 192],
        [180, 189, 198],
        ...,
        [107, 111, 106],
        [105, 108, 106],
        [ 88,  91,  89]],

       [[169, 177, 184],
        [149, 157, 164],
        [155, 163, 170],
        ...,
        [106, 106, 100],
        [108, 107, 103],
        [100, 101,  97]],

       [[149, 157, 164],
        [143, 151, 158],
        [136, 144, 151],
        ...,
        [109, 107,  

In [ ]:
# Load data
feature_matrix = pd.read_csv('/content/drive/MyDrive/data/feature_matrix.csv')
output_matrix = pd.read_csv('/content/drive/MyDrive/data/output_matrix.csv')

In [ ]:
output_matrix.head(5)

,Unnamed: 0,SINR,RSRP,RSRQ,Power
0,0,16.18,-75.44,-13.36,-55.42
1,1,16.18,-75.44,-13.36,-55.42
2,2,16.18,-75.44,-13.36,-55.42
3,3,16.18,-75.44,-13.36,-55.42
4,4,16.18,-75.44,-13.36,-55.42


In [ ]:
feature_matrix.head(5)

,Unnamed: 0,Longitude,Latitude,Speed,Distance,Distance_x,Distance_y,PCI,PCI_64,PCI_65,PCI_302
0,0,12.520112,55.780073,21.51,0.553064,-0.004597,-0.003369,64,1,0,0
1,1,12.520119,55.780071,20.93,0.553104,-0.004599,-0.003362,64,1,0,0
2,2,12.520127,55.780070,20.36,0.553016,-0.004600,-0.003354,64,1,0,0
3,3,12.520135,55.780068,19.78,0.553033,-0.004602,-0.003346,64,1,0,0
4,4,12.520142,55.780067,19.21,0.552970,-0.004603,-0.003339,64,1,0,0


In [ ]:
f_matrix = feature_matrix.head(50)

In [ ]:
o_matrix = output_matrix.head(50)

In [ ]:
# Concatenate along columns (axis=1) to join data side by side
path_loss_data = pd.concat([f_matrix, o_matrix], axis=1)

In [ ]:
path_loss_data.head()

,Unnamed: 0,Longitude,Latitude,Speed,Distance,Distance_x,Distance_y,PCI,PCI_64,PCI_65,PCI_302,Unnamed: 0,SINR,RSRP,RSRQ,Power
0,0,12.520112,55.780073,21.51,0.553064,-0.004597,-0.003369,64,1,0,0,0,16.18,-75.44,-13.36,-55.42
1,1,12.520119,55.780071,20.93,0.553104,-0.004599,-0.003362,64,1,0,0,1,16.18,-75.44,-13.36,-55.42
2,2,12.520127,55.780070,20.36,0.553016,-0.004600,-0.003354,64,1,0,0,2,16.18,-75.44,-13.36,-55.42
3,3,12.520135,55.780068,19.78,0.553033,-0.004602,-0.003346,64,1,0,0,3,16.18,-75.44,-13.36,-55.42
4,4,12.520142,55.780067,19.21,0.552970,-0.004603,-0.003339,64,1,0,0,4,16.18,-75.44,-13.36,-55.42


In [ ]:
# Filter and drop unnamed columns
unnamed_columns = path_loss_data.filter(regex='Unnamed').columns
path_loss_data = path_loss_data.drop(columns=unnamed_columns)

In [ ]:
path_loss_data.head()

,Longitude,Latitude,Speed,Distance,Distance_x,Distance_y,PCI,PCI_64,PCI_65,PCI_302,SINR,RSRP,RSRQ,Power
0,12.520112,55.780073,21.51,0.553064,-0.004597,-0.003369,64,1,0,0,16.18,-75.44,-13.36,-55.42
1,12.520119,55.780071,20.93,0.553104,-0.004599,-0.003362,64,1,0,0,16.18,-75.44,-13.36,-55.42
2,12.520127,55.780070,20.36,0.553016,-0.004600,-0.003354,64,1,0,0,16.18,-75.44,-13.36,-55.42
3,12.520135,55.780068,19.78,0.553033,-0.004602,-0.003346,64,1,0,0,16.18,-75.44,-13.36,-55.42
4,12.520142,55.780067,19.21,0.552970,-0.004603,-0.003339,64,1,0,0,16.18,-75.44,-13.36,-55.42


In [ ]:
path_loss_data.dtypes

Longitude     float64
Latitude      float64
Speed         float64
Distance      float64
Distance_x    float64
Distance_y    float64
PCI             int64
PCI_64          int64
PCI_65          int64
PCI_302         int64
SINR          float64
RSRP          float64
RSRQ          float64
Power         float64
dtype: object

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(loaded_images, path_loss_data, test_size=0.2, random_state=42)


In [ ]:
y_test.shape

(10, 14)

In [ ]:
# Load a sample image
sample_image = cv2.imread('/content/drive/MyDrive/data/train/9932.png')

# Get the shape of the image array
image_height, image_width, num_channels = sample_image.shape

print("Image Height:", image_height)
print("Image Width:", image_width)
print("Number of Channels:", num_channels)

Image Height: 256
Image Width: 256
Number of Channels: 3


In [ ]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer for path loss prediction
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


ValueError: Data cardinality is ambiguous:
  x sizes: 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256
  y sizes: 40
Make sure all arrays contain the same number of samples.